In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [4]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [6]:
%%sql
SELECT * FROM facts limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [10]:
%%sql
SELECT
    MIN(population) min_pop,
    MAX(population) max_pop,
    MIN(population_growth) min_pop_grwth,
    MAX(population_growth) max_pop_grwth
FROM facts;

Done.


min_pop,max_pop,min_pop_grwth,max_pop_grwth
0,7256490011,0.0,4.02


### Исследуем выбросы: max и min

In [11]:
%%sql
SELECT * FROM facts
WHERE population == (SELECT MAX(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


In [12]:
%%sql
SELECT * FROM facts
WHERE population == (SELECT MIN(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


### Изучение среднего населения и площади

In [15]:
%%sql
SELECT 
    round(AVG(population), 2), 
    round(AVG(area), 2) FROM facts;

Done.


"round(AVG(population), 2)","round(AVG(area), 2)"
62094928.32,555093.55


### Определение плотно населенных стран

In [17]:
%%sql
SELECT * FROM facts
WHERE population > (
    SELECT AVG(population)
    FROM facts)
AND area < (
    SELECT AVG(area)
    FROM facts) 

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


### Какие страны имеют самые высокие соотношения воды и земли?

In [19]:
%%sql
PRAGMA TABLE_INFO(facts);

Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,1,None,1
1,code,varchar(255),1,None,0
2,name,varchar(255),1,None,0
3,area,integer,0,None,0
4,area_land,integer,0,None,0
5,area_water,integer,0,None,0
6,population,integer,0,None,0
7,population_growth,float,0,None,0
8,birth_rate,float,0,None,0
9,death_rate,float,0,None,0


In [43]:
%%sql
SELECT name, CAST(area_water as float) / CAST(area_land as float) as ratios 
FROM facts
ORDER BY ratios DESC
LIMIT 10

Done.


name,ratios
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


### Какие страны добавят больше и меньше всего населения в следующем году?

In [53]:
%%sql
SELECT *, CAST(area_water as float) / CAST(area_land as float) as ratios 
FROM facts
WHERE ratios > 1.0


Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,ratios
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.6666666666666
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4.520231213872832


In [61]:
%%sql
select * from facts
where population_growth > (select AVG(population_growth) from facts)
order by population_growth desc
limit 10

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62


In [64]:
%%sql
select * from facts
where population_growth < (select AVG(population_growth) from facts)
order by population_growth
limit 10

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
203,nf,Norfolk Island,36,36,0,2210,0.01,None,None,None
221,tl,Tokelau,12,12,0,1337,0.01,None,None,None
232,fk,Falkland Islands (Islas Malvinas),12173,12173,0,3361,0.01,10.9,4.9,None
72,gy,Guyana,214969,196849,18120,735222,0.02,15.59,7.32,8.06
157,lo,Slovakia,49035,48105,930,5445027,0.02,9.91,9.74,0.04


### В каких странах уровень смертности выше, чем уровень рождаемости?

In [72]:
%%sql
select name, death_rate, birth_rate, round(death_rate/birth_rate, 2) as ratio
from facts
where death_rate > birth_rate
order by ratio desc


Done.


name,death_rate,birth_rate,ratio
Bulgaria,14.44,8.92,1.62
Serbia,13.66,9.08,1.5
Latvia,14.31,10.0,1.43
Lithuania,14.27,10.1,1.41
Hungary,12.73,9.16,1.39
Monaco,9.24,6.65,1.39
Germany,11.42,8.47,1.35
Slovenia,11.37,8.42,1.35
Ukraine,14.46,10.72,1.35
Saint Pierre and Miquelon,9.72,7.42,1.31


In [73]:
%%sql
select name, death_rate, birth_rate, round(death_rate/birth_rate, 2) as ratio
from facts
where death_rate < birth_rate
order by ratio
limit 10

Done.


name,death_rate,birth_rate,ratio
Gaza Strip,3.04,31.11,0.1
Kuwait,2.18,19.91,0.11
Iraq,3.77,31.45,0.12
United Arab Emirates,1.97,15.43,0.13
Oman,3.36,24.44,0.14
Jordan,3.79,25.37,0.15
Solomon Islands,3.85,25.77,0.15
West Bank,3.5,22.99,0.15
Marshall Islands,4.21,25.6,0.16
Qatar,1.53,9.84,0.16
